<a href="https://colab.research.google.com/github/Scottman625/Tibame-DeepLearning/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84140032/84125825 [==============================] - 9s 0us/step


In [ ]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [ ]:
train_df = getdata("train")
test_df = getdata("test")

In [ ]:
train_df

,content,sentiment
0,"Fabulous actors, beautiful scenery, stark real...",1
1,"This movie is humorous, charming, and easily b...",1
2,"""The Golden Child"" was Eddie Murphy's first fi...",1
3,The wonderfully urbane Ronald Coleman is show-...,1
4,"As a baseball die-hard, this movie goes contra...",1
...,...,...
24995,**** SPOILER WARNING ****<br /><br />Absolutel...,0
24996,"This movie was bizarre, completely inexplicabl...",0
24997,This movie was terrible. The first half hour i...,0
24998,The writers and producers of this little outin...,0


In [ ]:
# 1. tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [ ]:
# tok.index_word
# tok.word_index

In [ ]:
# 2. sequenize: 文字化成數字(數字最大值:3000)
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,2718,153,304,1381,632,10,525,20,29.0,4.0,1.0,82.0,792.0,85.0,22.0,76.0,1.0,428.0,187.0,1.0,17.0,215.0,15.0,1.0,632.0,6.0,2.0,82.0,8.0,657.0,5.0,2002.0,10.0,79.0,25.0,271.0,10.0,329.0,9.0,447.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,17,6,1975,1217,2,711,457,3.0,511.0,15.0,145.0,34.0,354.0,635.0,718.0,359.0,6.0,979.0,1.0,270.0,15.0,618.0,476.0,35.0,10.0,328.0,1773.0,9.0,14.0,6.0,8.0,24.0,641.0,402.0,176.0,70.0,14.0,1.0,1588.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2043,503,13,1807,83,19,234,24.0,998.0,2.0,57.0,148.0,42.0,21.0,14.0,49.0,14.0,998.0,42.0,3.0,250.0,695.0,1150.0,2683.0,378.0,14.0,3.0,4.0,413.0,473.0,868.0,3.0,88.0,1727.0,417.0,24.0,5.0,166.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1664,6,120,130,14,8,168,82.0,4.0,24.0,105.0,26.0,6.0,1224.0,8.0,172.0,133.0,2.0,11.0,2837.0,17.0,1284.0,1473.0,85.0,4.0,87.0,21.0,8.0,2706.0,4.0,87.0,26.0,6.0,2246.0,1906.0,2.0,52.0,1092.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,3,2354,781,251,11,17,268,5.0,48.0,10.0,532.0,8.0,3.0,2247.0,17.0,2888.0,264.0,2247.0,202.0,864.0,102.0,2.0,32.0,201.0,62.0,344.0,134.0,8.0,1.0,993.0,8.0,1.0,83.0,2373.0,9.0,2876.0,1289.0,8.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1358,1706,7,7,424,206,3,822,28.0,4.0,1.0,1529.0,1288.0,123.0,1072.0,15.0,1.0,265.0,481.0,1164.0,5.0,190.0,98.0,4.0,11.0,612.0,9.0,59.0,190.0,3.0,675.0,664.0,5.0,1027.0,29.0,4.0,1.0,209.0,8.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,11,17,13,1158,337,2,5,103,16.0,365.0,134.0,2775.0,8.0,3.0,191.0,1893.0,310.0,10.0,63.0,116.0,1.0,633.0,535.0,16.0,2750.0,41.0,413.0,8.0,3.0,1487.0,540.0,10.0,178.0,5.0,27.0,3.0,304.0,413.0,8.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,17,13,391,1,83,317,531,6.0,73.0,37.0,3.0,70.0,15.0,1.0,580.0,4.0,18.0,9.0,13.0,328.0,3.0,75.0,307.0,4.0,3.0,1.0,83.0,133.0,6.0,217.0,36.0,28.0,4.0,1.0,83.0,8.0,1501.0,9.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,1,925,2,1179,4,11,114,25,159.0,4.0,119.0,267.0,8.0,35.0,910.0,39.0,66.0,1007.0,53.0,35.0,73.0,12.0,33.0,68.0,915.0,5.0,1470.0,3.0,2311.0,133.0,118.0,3.0,182.0,252.0,8.0,1.0,142.0,2220.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 3. padding: 變成一樣長
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,371,1127,12,432,43,4,967,18,1,285,6,147,15,10,9,20,2353,21,702,18,70,287,9,5,69,11,6,3,17,10,77,27,146,363,1,127,4,58,501
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,642,21,5,757,24,1023,30,628,7,7,1,62,6,21,930,18,42,70,221,8,58,649,2,10,354,9,50,254,55,10,64,9,42,84,220,718,96,7,7
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17,97,25,74,998,2,1,4,234,528,4,11,17,295,37,3,1150,1927,37,32,1525,209,42,185,209,16,916,2,1150,16,315,299,772,4,250,7,7,43,4,685
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15,11,52,160,19,1,774,440,6,3,2218,4,2044,2,2588,14,26,2665,3,1484,1400,4,1,82,102,24,1951,67,27,1033,2521,8,1,353,413,60,172,19,212,64
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15,1583,65,809,184,2,2471,2,1462,1650,166,139,5,261,8,8,840,405,3,324,236,2,1220,392,3,53,79,124,70,463,692,257,1453,323,4,86,5,43,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,28,5,76,8,1444,16,95,312,1,474,385,1,274,133,16,2376,1262,20,3,8,1,24,704,8,1,17,967,6,32,353,26,66,955,5,41,26,112,119,36,11
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,282,89,103,11,44,22,162,178,111,39,39,229,30,29,5,94,278,181,304,148,22,200,40,199,53,20,11,2,1194,5,103,32,776,186,17,37,132,348,1236
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,188,76,117,9,3,50,422,59,25,74,150,4,564,39,57,125,3,7,7,40,78,620,3,2075,2,795,11,17,44,22,58,2009,2,190,9,43,550,1682,69
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,164,465,5,25,32,773,289,8,11,9,183,12,1,61,455,26,212,25,345,13,7,7,2,14,15,1,1932,44,22,103,11,588,27,249,5,25,46,30,1,1621


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

layers = [
    Embedding(3001, 128, mask_zero=True, input_length=512),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 128)          384128    
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 不用做one-hot
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [ ]:
# 1 round: 54000 / 100 -> 540次梯度下降
# verbose: 0(quiet) 1(full) 2(no progress bar)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("imdb.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad,
          y_train,
          batch_size=100,
          epochs=100,
          verbose=2,
          validation_split=0.1,
          callbacks=callbacks)

Epoch 1/100
225/225 - 6s - loss: 0.5715 - accuracy: 0.7158 - val_loss: 0.4821 - val_accuracy: 0.7900 - 6s/epoch - 26ms/step
Epoch 2/100
225/225 - 5s - loss: 0.3623 - accuracy: 0.8641 - val_loss: 0.3755 - val_accuracy: 0.8376 - 5s/epoch - 23ms/step
Epoch 3/100
225/225 - 5s - loss: 0.2927 - accuracy: 0.8869 - val_loss: 0.3768 - val_accuracy: 0.8304 - 5s/epoch - 22ms/step
Epoch 4/100
225/225 - 5s - loss: 0.2619 - accuracy: 0.8980 - val_loss: 0.3177 - val_accuracy: 0.8688 - 5s/epoch - 23ms/step
Epoch 5/100
225/225 - 5s - loss: 0.2431 - accuracy: 0.9057 - val_loss: 0.2762 - val_accuracy: 0.8912 - 5s/epoch - 23ms/step
Epoch 6/100
225/225 - 5s - loss: 0.2310 - accuracy: 0.9090 - val_loss: 0.3352 - val_accuracy: 0.8640 - 5s/epoch - 23ms/step
Epoch 7/100
225/225 - 5s - loss: 0.2222 - accuracy: 0.9146 - val_loss: 0.2847 - val_accuracy: 0.8908 - 5s/epoch - 22ms/step
Epoch 8/100
225/225 - 5s - loss: 0.2144 - accuracy: 0.9174 - val_loss: 0.3131 - val_accuracy: 0.8788 - 5s/epoch - 23ms/step
Epoch 9/

In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.2866 - accuracy: 0.8814


[0.28663936257362366, 0.8813999891281128]

In [ ]:
text = "I've seen ever spider man movie and this one is the worst in my opinion. It is not edgy at all but rather just goofy silly as if made for young children. Zendaya is completely uninteresting as a love interest. There is zero chemistry between her and Tom holland. Dr strange is portrayed as a bumbling idiot. Just stupid overall."#@param {type:"string"}
# sequenize
seq = tok.texts_to_sequences([text])
seq_pad = pad_sequences(seq, maxlen=512)
proba = model.predict(seq_pad)[0]
trans = ["neg", "pos"]
for t, p in zip(trans, proba):
    print(t, "的機率:", p)

neg 的機率: 0.9988734
pos 的機率: 0.0011265744


In [ ]:
from tensorflow.keras.models import Sequential
layers = [
    Embedding(3001, 128, mask_zero=True), 
    GlobalAveragePooling1D()
]
partial = Sequential(layers)
partial.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         384128    
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
w = model.layers[0].get_weights()
partial.layers[0].set_weights(w)

In [ ]:
from scipy.spatial.distance import cosine
words1 = "Accension is go to higher dimension."#@param {type:"string"}
seq = tok.texts_to_sequences([words1])
vector1 = partial.predict(seq)[0]
words2 = "Enlightment is to realize the truth."#@param {type:"string"}
seq = tok.texts_to_sequences([words2])
vector2 = partial.predict(seq)[0]
# cos: 1(正向相似) 0(不相干) -1(反向相似)
# 絕對值0.5以上, 代表兩個情緒蠻相似的
print("cos value:", 1 - cosine(vector1, vector2))

cos value: 0.6479371786117554


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.layers import SimpleRNN

layers = [
    # 3001(種) * 128(情緒) -> 384128
    Embedding(3001, 128, mask_zero=True, input_length=512),
    # 目前embedding(128 * 64) + 前一個(64 * 64) +64(bias) = 12352
    SimpleRNN(64),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 512, 128)          384128    
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 396,610
Trainable params: 396,610
Non-trainable params: 0
_________________________________________________________________
